# KERAS LAYERED MODEL (CNN)

## CONTENTS:

1. Import Libraries and Data¶
2. Data Wrangling and Reshaping
3. Data Splitting
4. Keras Model Creation
5. Compiling and Running Model
6. Confusion Matrix Creation
7. Keras Model Retrials (Until Convergence)

### 1. IMPORT LIBRARIES AND DATA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define path for where data is stored
path = r'C:\Users\Administrator\Documents\data analytics\Machine Learning\ClimateWins\Data Sets'

In [3]:
# Import the weather data that was already scaled. 
climate = pd.read_csv(os.path.join(path, 'data_scaled.csv'))

In [4]:
climate.head(3)

,Unnamed: 0,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,0,-1.599964,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,...,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237,19600101
1,1,-1.599964,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,...,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141,19600102
2,2,-1.599964,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,...,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078,19600103


In [5]:
# Drop the unnecessary unnamed column
climate= climate.drop('Unnamed: 0', axis = 1)

In [6]:
climate.head(3)

,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,...,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,-1.599964,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,-0.528623,...,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237,19600101
1,-1.599964,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,-0.582946,...,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141,19600102
2,-1.599964,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,-0.257010,...,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078,19600103


In [7]:
climate.shape

(22950, 170)

In [8]:
# Read in the Answers data.
answers = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [9]:
answers.shape

(22950, 16)

### 2. DATA WRANGLING AND RESHAPING

In [10]:
# Drop Gdansk, Roma and Tours since they are missing from the Answers file. I use a filtering function. 

# List of city names
cities = ['GDANSK', 'ROMA', 'TOURS']

# Find columns that include any of the city names
cols_to_drop = [col for col in climate.columns if any(city in col for city in cities)]

# Drop those columns
climatedr = climate.drop(columns=cols_to_drop)

# Count nr of columns in climatedr
num_columns = climatedr.shape[1]
print(num_columns)

149


In [11]:
climatedr.shape

(22950, 149)

In [12]:
climatedr.head(3)

,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,...,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,-1.599964,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,-0.528623,...,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237,19600101
1,-1.599964,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,-0.582946,...,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141,19600102
2,-1.599964,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,-0.257010,...,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078,19600103


In [13]:
climatedr.isnull().sum()

MONTH                 0
BASEL_cloud_cover     0
BASEL_wind_speed      0
BASEL_humidity        0
BASEL_pressure        0
                     ..
VALENTIA_sunshine     0
VALENTIA_temp_mean    0
VALENTIA_temp_min     0
VALENTIA_temp_max     0
DATE                  0
Length: 149, dtype: int64

In [14]:
# Extract the different observation types

observation_types = ['cloud_cover', 'wind_speed', 'humidity', 'pressure',
                     'global_radiation', 'precipitation', 'snow_depth', 
                     'sunshine', 'temp_mean', 'temp_min', 'temp_max']

In [15]:
# Create a dictionary to store the count of stations for each observation type
station_counts = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in climatedr.columns if col.endswith(obs)]
    
    # Count the number of stations (i.e., the number of columns) for the current observation type
    station_counts[obs] = len(columns)

# Print the count of stations for each observation type
print("Number of stations covered by each observation type:")
for obs, count in station_counts.items():
    print(f"{obs}: {count} stations")


Number of stations covered by each observation type:
cloud_cover: 14 stations
wind_speed: 9 stations
humidity: 14 stations
pressure: 14 stations
global_radiation: 15 stations
precipitation: 15 stations
snow_depth: 6 stations
sunshine: 15 stations
temp_mean: 15 stations
temp_min: 15 stations
temp_max: 15 stations


Windspeed and snowdepth are missing entries and will be dropped. 

In [16]:
# Get a list of columns containing 'wind_speed' or 'snow_depth'
cols_to_drop = [col for col in climatedr.columns if '_wind_speed' in col or '_snow_depth' in col]

# Drop the columns
climate2 = climatedr.drop(cols_to_drop, axis=1)

In [17]:
climate2.shape # 15 columns less since they were dropped.

(22950, 134)

One entry missing for cloud humidity, pression and cloud cover. 

In [18]:
# Find the stations with the above entries missing
# Get all column names
all_columns = climate2.columns.tolist()
# Exclude 'DATE' and 'MONTH' columns
all_columns = [col for col in all_columns if col not in ['DATE', 'MONTH']]  
# Extract unique weather station names
weather_stations = set()  # Use a set to automatically store only unique values
for col in all_columns:
    station_name = col.split('_')[0]  # Split the column name at the underscore and take the first part
    weather_stations.add(station_name)

# Print the list of weather stations
print(weather_stations)

{'SONNBLICK', 'DUSSELDORF', 'BELGRADE', 'KASSEL', 'STOCKHOLM', 'MAASTRICHT', 'DEBILT', 'BASEL', 'HEATHROW', 'MADRID', 'VALENTIA', 'LJUBLJANA', 'MUNCHENB', 'OSLO', 'BUDAPEST'}


In [19]:
# Find stations missing observation types
observation_types = ['cloud_cover', 'humidity', 'pressure']

missing_stations_by_observation = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in climate2.columns if col.endswith(obs)]
    
    # Extract station names by removing the observation type from the column names
    station_names = set([col.replace(f'_{obs}', '') for col in columns])
    
    # Identify stations that are in all_stations but missing from the current observation type
    missing_stations = weather_stations - station_names
    
    # Store the missing station names in the dictionary
    missing_stations_by_observation[obs] = missing_stations

# Print the missing station names for each observation type
for obs, missing_stations in missing_stations_by_observation.items():
    print(f"\nStations missing from {obs}:")
    if missing_stations:
        for station in missing_stations:
            print(station)
    else:
        print("None")


Stations missing from cloud_cover:
KASSEL

Stations missing from humidity:
STOCKHOLM

Stations missing from pressure:
MUNCHENB


In [20]:
climate2.head(3)

,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,DATE
0,-1.599964,0.660514,0.826097,-0.001949,-1.101066,-0.265148,-0.902918,-0.528623,-0.845652,-0.478356,...,-0.443701,0.761754,-1.299744,-0.806427,-0.088407,0.372147,-0.668215,-0.519743,-0.752237,19600101
1,-1.599964,0.244897,0.735760,-0.001949,-1.058108,1.658760,-0.810126,-0.582946,-0.462450,-0.569988,...,0.783085,1.183580,-1.262455,-1.042055,0.503361,-0.829285,-0.548046,-0.629054,-0.407141,19600102
2,-1.599964,1.076130,1.277781,-0.001949,-1.251420,0.155707,-1.065304,-0.257010,-0.186545,-0.592896,...,0.783085,1.183580,-0.432779,-1.136306,-0.396127,-1.009500,-0.067372,0.054135,-0.177078,19600103


In [21]:
# Cloud cover is the start of a stations data, Kassel is next to Heathrow, find the position of Heathrow_temp_max for the insertion of Kassel_cloud_cover
climate2.columns.get_loc('HEATHROW_temp_max')

54

In [22]:
# Find the position for insertion of Stockholm humidity
climate2.columns.get_loc('STOCKHOLM_cloud_cover') #humidity is 1 after cloud cover so (result +1)

116

In [23]:
# Find position for Munchenb pressure
climate2.columns.get_loc('MUNCHENB_cloud_cover') # pressure is 2 after cloud cover so (result +2)

90

In [24]:
# Insert new columns into "unscaled" at specific positions.
# The data for these new columns is taken from weather stations they are close to

climate2.insert(56,'KASSEL_cloud_cover', climate2['DUSSELDORF_cloud_cover'])
climate2.insert(119, 'STOCKHOLM_humidity', climate2['OSLO_humidity'])
climate2.insert(94,'MUNCHENB_pressure',climate2['BASEL_pressure'])

In [25]:
climate2.columns.tolist()

['MONTH',
 'BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSSELDORF_te

In [26]:
climate2.shape

(22950, 137)

In [27]:
# Drop unnecessary columns
climate2.drop(['DATE', 'MONTH'], axis=1, inplace=True)

In [28]:
# confirm drop
climate2.shape

(22950, 135)

In [29]:
answers.head(3)

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
# drop unneeded column from second dataset
answers.drop(columns = 'DATE', inplace = True)

In [31]:
# check drop
answers.head(3)

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
answers.shape

(22950, 15)

In [33]:
# Export cleaned dataset
climate2.to_csv(os.path.join(path, 'climate_clean.csv'), index=False)

2.1 DATA RESHAPING 

In [34]:
# Create an 'X' matrix by reloading and naming our data 'X'
x=pd.read_csv(os.path.join(path, 'climate_clean.csv'), index_col=False)

In [35]:
x.head(3)

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,0.660514,0.826097,-0.001949,-1.101066,-0.265148,-0.902918,-0.528623,-0.845652,-0.478356,-1.206433,...,-0.639538,-0.443701,0.761754,-1.299744,-0.806427,-0.088407,0.372147,-0.668215,-0.519743,-0.752237
1,0.244897,0.735760,-0.001949,-1.058108,1.658760,-0.810126,-0.582946,-0.462450,-0.569988,0.652846,...,-0.628550,0.783085,1.183580,-1.262455,-1.042055,0.503361,-0.829285,-0.548046,-0.629054,-0.407141
2,1.076130,1.277781,-0.001949,-1.251420,0.155707,-1.065304,-0.257010,-0.186545,-0.592896,0.652846,...,-0.727444,0.783085,1.183580,-0.432779,-1.136306,-0.396127,-1.009500,-0.067372,0.054135,-0.177078


In [36]:
y = answers

In [37]:
x.shape

(22950, 135)

In [43]:
# Turn X and y into arrays
x = np.array(x)
y = np.array(y)
x

array([[ 6.60513663e-01,  8.26096599e-01, -1.94863388e-03, ...,
        -6.68214979e-01, -5.19743407e-01, -7.52236990e-01],
       [ 2.44896945e-01,  7.35759689e-01, -1.94863388e-03, ...,
        -5.48046319e-01, -6.29053523e-01, -4.07141387e-01],
       [ 1.07613038e+00,  1.27778115e+00, -1.94863388e-03, ...,
        -6.73716818e-02,  5.41347039e-02, -1.77077651e-01],
       ...,
       [-5.86336492e-01,  1.30644098e-02,  7.16401992e-01, ...,
        -7.28735214e-03, -5.20354258e-04, -4.52984969e-03],
       [-1.70719774e-01,  3.74412049e-01,  4.87141154e-01, ...,
        -7.28735214e-03, -5.20354258e-04, -4.52984969e-03],
       [-1.70719774e-01,  7.35759689e-01,  1.96744092e-01, ...,
        -7.28735214e-03, -5.20354258e-04, -4.52984969e-03]])

In [44]:
x = x.reshape(-1,15,9)

In [45]:
# Verify Shape
x.shape

(22950, 15, 9)

In [46]:
# Verify Shape
y.shape

(22950, 15)

In [47]:
x

array([[[ 6.60513663e-01,  8.26096599e-01, -1.94863388e-03, ...,
         -5.28623012e-01, -8.45651922e-01, -4.78356271e-01],
        [-1.20643263e+00,  9.05270489e-01,  3.21770762e-01, ...,
         -1.01687613e+00, -1.22021042e+00, -9.49202784e-01],
        [-2.55240242e-01, -1.00976762e-02, -5.87602906e-03, ...,
         -1.09916317e+00, -1.11943125e+00, -1.13683915e+00],
        ...,
        [-4.28835402e-01, -6.29430381e-01,  9.52340493e-02, ...,
         -1.24330511e-01, -1.70125111e-01, -6.33220094e-02],
        [-6.01023076e-02, -3.46465175e-03,  1.54402321e+00, ...,
         -3.91072163e-01, -2.90438548e-01, -6.39537957e-01],
        [-4.43700743e-01,  7.61754381e-01, -1.29974368e+00, ...,
         -6.68214979e-01, -5.19743407e-01, -7.52236990e-01]],

       [[ 2.44896945e-01,  7.35759689e-01, -1.94863388e-03, ...,
         -5.82945633e-01, -4.62450189e-01, -5.69988328e-01],
        [ 6.52845676e-01,  1.11911758e+00,  1.68378867e-02, ...,
         -1.10766931e+00, -8.18701592e

### 3. DATA SPLITTING

In [48]:
# Split data into train and test sets

x_train, x_test, y_train, y_test = train_test_split(x,y,random_state = 42)

In [49]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


### 4. KERAS MODEL CREATION

In [50]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [51]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 14, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,831 (11.06 KB)

 Trainable params: 2,831 (11.06 KB)

 Non-trainable params: 0 (0.00 B)

### 5. COMPILING AND RUNNING MODEL

In [52]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [53]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1334 - loss: 1904.4594
Epoch 2/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1470 - loss: 19126.7090
Epoch 3/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1574 - loss: 64848.9922
Epoch 4/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1561 - loss: 144413.9219
Epoch 5/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1503 - loss: 234221.6719
Epoch 6/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1508 - loss: 367186.4688
Epoch 7/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1464 - loss: 547163.0625
Epoch 8/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1523 - loss: 742340.5625
Epoch 9/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1495 - loss: 976095.1250
Epoch 10/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1513 - loss: 1252263.8750
Epoch 11/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1479 - loss: 1597304.7500
Epoch 12/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1515 - loss: 1970624.8750
Epoch 13/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1534 - loss: 2422371.0000
Epoch 14

### 6.CONFUSION MATRIX CREATION

In [54]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [55]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [56]:
# Evaluate
print(confusion_matrix(y_test, model.predict(x_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         107        20       966     102         939        46     195   
BELGRADE        0         8       792       9         240         5       6   
BUDAPEST        0         1        90       6          95         1       5   
DEBILT          0         0         8       1          68         0       4   
DUSSELDORF      0         0         3       3          21         0       0   
HEATHROW        0         0        15       3          42         3       4   
KASSEL          0         0         3       0           8         0       0   
LJUBLJANA       1         0        18       0          13         0       4   
MAASTRICHT      0         0         1       2           6         0       0   
MADRID          0         2       108      10          75         7      22   
MUNCHENB 

In [ ]:
7. KERAS MODEL RETRIALS

In [58]:
epochs = 60
batch_size = 16
n_hidden = 32

timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [59]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [60]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1235 - loss: 1410.6365
Epoch 2/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1608 - loss: 13877.4824
Epoch 3/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1646 - loss: 44752.8359
Epoch 4/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1627 - loss: 96246.7891
Epoch 5/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1592 - loss: 174258.3281
Epoch 6/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1580 - loss: 278343.1875
Epoch 7/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1552 - loss: 404655.9062
Epoch 8/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1534 - loss: 570737.1875
Epoch 9/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1559 - loss: 765739.9375
Epoch 10/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1531 - loss: 989759.6250
Epoch 11/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1534 - loss: 1261195.7500
Epoch 12/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1500 - loss: 1571485.2500
Epoch 13/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1505 - loss: 1927498.5000
Epoch 14/6

In [61]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [62]:
# Evaluate
print(confusion_matrix(y_test, model.predict(x_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 969us/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         165       435       652     110         620       347      40   
BELGRADE        0       205       418       5         131        71       3   
BUDAPEST        1        18        44      10          29        31       1   
DEBILT          0         1         6       3          24        27       3   
DUSSELDORF      0         0         6       2           6        10       0   
HEATHROW        0         2         2       4          11        31       0   
KASSEL          0         0         4       0           5         1       0   
LJUBLJANA       3         4        10       0           4         4       1   
MAASTRICHT      0         0         2       1           3         0       0   
MADRID          9        30        23       7          14        51       3   
MUNCHENB 

Softmax is not producing good results. Trials with tanh, relu and sigmoid.

In [63]:
epochs = 30
batch_size = 16
n_hidden = 128

timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [64]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 14, 128)        │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 14, 16)         │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,191 (24.18 KB)

 Trainable params: 6,191 (24.18 KB)

 Non-trainable params: 0 (0.00 B)

In [65]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [67]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 3s - 2ms/step - accuracy: 0.1016 - loss: 25.0849
Epoch 2/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0448 - loss: 26.2683
Epoch 3/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0378 - loss: 26.0808
Epoch 4/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0296 - loss: 25.9741
Epoch 5/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0231 - loss: 24.9803
Epoch 6/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0330 - loss: 25.6493
Epoch 7/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0325 - loss: 26.1002
Epoch 8/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0347 - loss: 26.0821
Epoch 9/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0287 - loss: 24.9244
Epoch 10/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0240 - loss: 26.3088
Epoch 11/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0332 - loss: 26.3695
Epoch 12/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0339 - loss: 26.4490
Epoch 13/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0332 - loss: 26.4480
Epoch 14/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0315 - l

In [68]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [70]:
# Evaluate

print(confusion_matrix(y_test, model.predict(x_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step
Pred        BUDAPEST  DUSSELDORF  LJUBLJANA  MAASTRICHT  MADRID  MUNCHENB  \
True                                                                        
BASEL           3362           6        122           0     189         1   
BELGRADE         939           5         54           0      89         0   
BUDAPEST         177           0         15           1      21         0   
DEBILT            79           0          1           0       2         0   
DUSSELDORF        24           0          4           0       1         0   
HEATHROW          73           0          2           0       7         0   
KASSEL            10           0          0           0       1         0   
LJUBLJANA         46           0          1           0      12         0   
MAASTRICHT         7           0          0           0       2         0   
MADRID           339           7         27           1      82         0   
MUNCHENB           5           0 

Better loss but low accuracy. Try adjust layers.

In [72]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [73]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [74]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 3s - 3ms/step - accuracy: 0.0712 - loss: 25.7615
Epoch 2/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0641 - loss: 25.8067
Epoch 3/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0546 - loss: 25.5359
Epoch 4/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0814 - loss: 25.7126
Epoch 5/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1474 - loss: 26.5583
Epoch 6/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1363 - loss: 25.0863
Epoch 7/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1349 - loss: 22.6069
Epoch 8/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1228 - loss: 23.5117
Epoch 9/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1027 - loss: 23.9153
Epoch 10/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0734 - loss: 26.1976
Epoch 11/30
1076/1076 - 2s - 2ms/step - accuracy: 0.0476 - loss: 25.4148
Epoch 12/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1035 - loss: 23.6871
Epoch 13/30
1076/1076 - 2s - 1ms/step - accuracy: 0.0670 - loss: 23.2646
Epoch 14/30
1076/1076 - 1s - 1ms/step - accuracy: 0.0530 - l

In [77]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [79]:
# Evaluate

print(confusion_matrix(y_test, model.predict(x_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 980us/step
Pred        BELGRADE  BUDAPEST  HEATHROW  LJUBLJANA  MAASTRICHT  OSLO  \
True                                                                    
BASEL           3258         1        56         50           6   211   
BELGRADE         920         0        19         18           0   109   
BUDAPEST         166         0         3          5           0    34   
DEBILT            69         0         1          4           0     8   
DUSSELDORF        23         0         0          1           0     4   
HEATHROW          69         0         0          2           0     9   
KASSEL            10         0         0          0           0     1   
LJUBLJANA         47         0         1          0           0     7   
MAASTRICHT         8         0         0          0           0     1   
MADRID           402         0         4          7           2    20   
MUNCHENB           8         0         0          0           0     0   
OSLO    

Higher accuracy but not stable loss. 

In [80]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) # Options: sigmoid, tanh, softmax, relu

In [81]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)               │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [84]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 2s - 2ms/step - accuracy: 0.5690 - loss: 3804.8333
Epoch 2/30
1076/1076 - 2s - 1ms/step - accuracy: 0.6434 - loss: 37306.3398
Epoch 3/30
1076/1076 - 2s - 1ms/step - accuracy: 0.6439 - loss: 114980.4844
Epoch 4/30
1076/1076 - 2s - 1ms/step - accuracy: 0.6438 - loss: 239309.5000
Epoch 5/30
1076/1076 - 2s - 2ms/step - accuracy: 0.6439 - loss: 423855.7500
Epoch 6/30
1076/1076 - 2s - 2ms/step - accuracy: 0.6439 - loss: 637136.0625
Epoch 7/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6439 - loss: 922681.8750
Epoch 8/30
1076/1076 - 2s - 1ms/step - accuracy: 0.6439 - loss: 1282413.0000
Epoch 9/30
1076/1076 - 2s - 2ms/step - accuracy: 0.6439 - loss: 1729257.8750
Epoch 10/30
1076/1076 - 2s - 2ms/step - accuracy: 0.6439 - loss: 2180966.5000
Epoch 11/30
1076/1076 - 2s - 2ms/step - accuracy: 0.6438 - loss: 2757055.7500
Epoch 12/30
1076/1076 - 2s - 1ms/step - accuracy: 0.6439 - loss: 3380462.7500
Epoch 13/30
1076/1076 - 1s - 1ms/step - accuracy: 0.6440 - loss: 4160569.2500
Epoch

In [85]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [86]:
# Evaluate

print(confusion_matrix(y_test, model.predict(x_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Pred        BASEL  BELGRADE
True                       
BASEL        3681         1
BELGRADE     1092         0
BUDAPEST      214         0
DEBILT         82         0
DUSSELDORF     29         0
HEATHROW       82         0
KASSEL         11         0
LJUBLJANA      61         0
MAASTRICHT      9         0
MADRID        458         0
MUNCHENB        8         0
OSLO            5         0
STOCKHOLM       4         0
VALENTIA        1         0


Not good loss and accuracy.

In [87]:
epochs = 15
batch_size = 4
n_hidden = 4

timesteps = len(x_train[0])
input_dim = len(x_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='relu')) # Options: sigmoid, tanh, softmax, relu

In [88]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 14, 16)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,851 (7.23 KB)

 Trainable params: 1,851 (7.23 KB)

 Non-trainable params: 0 (0.00 B)

In [89]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [91]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/15
4303/4303 - 7s - 2ms/step - accuracy: 0.0929 - loss: 9.4072
Epoch 2/15
4303/4303 - 5s - 1ms/step - accuracy: 0.1364 - loss: 7.9650
Epoch 3/15
4303/4303 - 5s - 1ms/step - accuracy: 0.1419 - loss: 7.8505
Epoch 4/15
4303/4303 - 6s - 1ms/step - accuracy: 0.5213 - loss: nan
Epoch 5/15
4303/4303 - 5s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/15
4303/4303 - 5s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/15
4303/4303 - 6s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/15
4303/4303 - 5s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/15
4303/4303 - 5s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/15
4303/4303 - 6s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/15
4303/4303 - 5s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 12/15
4303/4303 - 5s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 13/15
4303/4303 - 6s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 14/15
4303/4303 - 5s - 1ms/step - accuracy: 0.6440 - loss: nan
Epoch 15/15
4303/4303 - 6s - 1ms/s

In [92]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [94]:
# Evaluate

print(confusion_matrix(y_test, model.predict(x_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step
Pred        BASEL
True             
BASEL        3682
BELGRADE     1092
BUDAPEST      214
DEBILT         82
DUSSELDORF     29
HEATHROW       82
KASSEL         11
LJUBLJANA      61
MAASTRICHT      9
MADRID        458
MUNCHENB        8
OSLO            5
STOCKHOLM       4
VALENTIA        1
